In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.schema.runnable import RunnablePassthrough


from langchain_community.cache import SQLiteCache
import langchain


chat = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# 캐시 설정
langchain.cache = SQLiteCache(database_path="./cache/langchain.db")


memory = ConversationBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
    memory_key="history"
)
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(docs, embeddings)

retriver = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | chat

def docs_to_context(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ask(question):
    docs = retriver.invoke(question)
    context = docs_to_context(docs)
    result = chain.invoke({"question": question, "context": context})
    memory.save_context({"input": question}, {"output": result.content})
    return result.content



/var/folders/tv/8v89wklj76j0w32lb_5dfpdc0000gn/T/ipykernel_46485/3757143375.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [2]:
ask("Is Aaronson guilty?")



According to Winston, Aaronson is guilty of the crimes he was charged with, but he also acknowledges that he had never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it can be inferred that Aaronson is not actually guilty, but the Party claims he is.

'According to Winston, Aaronson is guilty of the crimes he was charged with, but he also acknowledges that he had never seen the photograph that disproved their guilt and that it had never existed; he had invented it. Therefore, it can be inferred that Aaronson is not actually guilty, but the Party claims he is.'

In [4]:
ask("What message did he write in the table?")


Winston wrote "FREEDOM IS SLAVERY" and beneath it "TWO AND TWO MAKE FIVE." He also wrote "GOD IS POWER."

'Winston wrote "FREEDOM IS SLAVERY" and beneath it "TWO AND TWO MAKE FIVE." He also wrote "GOD IS POWER."'

In [5]:
ask("Who is Julia?")

Julia is a character who has a romantic relationship with Winston. She is a fellow Party member who initially appears to conform to the Party's rules but secretly rebels against it. Julia represents a sense of personal freedom and desire in contrast to the oppressive regime of the Party.

"Julia is a character who has a romantic relationship with Winston. She is a fellow Party member who initially appears to conform to the Party's rules but secretly rebels against it. Julia represents a sense of personal freedom and desire in contrast to the oppressive regime of the Party."